In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_today_date():
    from datetime import datetime
    date_str = str(datetime.now())
    today=datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    return today.strftime('%d-%b-%Y')

def get_index_investing_dot_com():
    url='https://in.investing.com/indices/major-indices'
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'   
    }

    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'html.parser')
    section=soup.find('section',class_="instrument")
    index={}
    for i in section.find_all('tr',class_='common-table-item'):
        index[i.find("td",class_="col-name").text.strip()]={"timestamp":i.find("td",class_="col-time").text.strip(),"last":i.find("td",class_="col-last").text.strip(),
                                                    "change":i.find("td",class_="col-chg").text.strip(),"pChange":i.find("td",class_="col-chg_pct").text.strip()}
    df=pd.DataFrame(index)
    df1=df[["Nifty 50","Nifty Bank","India VIX","Dow Jones","DAX"]]
    return df1.T
def get_futures_investing_dot_com():
    url='https://in.investing.com/indices/indices-futures'
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'   
    }

    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'html.parser')
    section=soup.find('section',class_="instrument")
    index={}
    for i in section.find_all('tr',class_='common-table-item'):
        index[i.find("td",class_="col-name").text.strip()]={"timestamp":i.find("td",class_="col-time").text.strip(),"last":i.find("td",class_="col-last").text.strip(),
                                                    "change":i.find("td",class_="col-chg").text.strip(),"pChange":i.find("td",class_="col-chg_pct").text.strip()}
    df=pd.DataFrame(index)
    df1=df[["Nifty 50","Bank NIFTY","Dow Jones"]]
    return df1.T

def red_font_negatives(series):
    highlight = 'color: red;'#,'font-weight: bold;'
    default = 'color: green;'
    return [highlight if e < 0 else default for e in series]

index=get_index_investing_dot_com()
futures=get_futures_investing_dot_com()
today=get_today_date()
print("Index details-",today)  
index["change"]=index["change"].astype(float)
display(index.style.apply(red_font_negatives, axis=0, subset=['change']).format(precision=2))

# display(get_index_investing_dot_com())
print("\nFutures details-",today)  
futures["change"]=futures["change"].astype(float)
display(futures.style.apply(red_font_negatives, axis=0, subset=['change']).format(precision=2))

# display(get_futures_investing_dot_com())

Index details- 10-Oct-2022


,timestamp,last,change,pChange
Nifty 50,09:17:41,"17,101.70",-212.95,-1.23%
Nifty Bank,09:17:41,"38,594.20",-583.85,-1.49%
India VIX,09:17:40,20.0350,1.22,+6.50%
Dow Jones,08/10,"29,296.79",-630.15,-2.11%
DAX,07/10,"12,273.00",-197.78,-1.59%



Futures details- 10-Oct-2022


,timestamp,last,change,pChange
Nifty 50,09:17:44,"17,073.00",-255.85,-1.48%
Bank NIFTY,07/10,"39,310.00",-68.25,-0.17%
Dow Jones,09:17:45,"29,191.6",-105.20,-0.36%


## Gainers

In [11]:
while True:
    try:
        def get_index_status_yf():
            def get_short_name(name):
                # 
                s1=name.split(' ')
                edit =s1[1]+' '+s1[2]
                return edit
           
            
            url='https://query2.finance.yahoo.com/v10/finance/quoteSummary/YM=F?formatted=true&crumb=XcWM00%2FDR.l&lang=en-US&region=US&modules=price%2CsummaryDetail%2CpageViews%2CfinancialsTemplate&corsDomain=finance.yahoo.com'
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}

            r_index = requests.get(url,headers=headers)
            index_data=r_index.json()
            item = index_data['quoteSummary']["result"][0]['price']
            dict1={}
            s=item["shortName"]
            shortName=get_short_name(s)#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<nested function

            # dict1[item["shortName"]]={'timestamp':item['regularMarketTime'],
            #                                               'last':item['regularMarketPrice']['fmt'],'variation':item['regularMarketChange']['fmt'],
            #                                               'percentChange':item['regularMarketChangePercent']['fmt']}
            dict1[shortName]={'timestamp':item['regularMarketTime'],
                                                          'last':item['regularMarketPrice']['fmt'],'variation':item['regularMarketChange']['fmt'],
                                                          'percentChange':item['regularMarketChangePercent']['fmt']}
            df=pd.DataFrame(dict1).T
            df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
            df1=df[['timestamp','last','variation','percentChange']]
            return df1
    
        def get_index_status_nse():
            url='https://www.nseindia.com/api/allIndices'
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}

            r_index = requests.get(url,headers=headers)
            index_data=r_index.json()
            item = index_data['data']
            dict={}
            dict1={}
            for i in range(len(item )):
                    dict[index_data['data'][i]['indexSymbol']]={'timestamp':index_data['timestamp'],
                                                          'last':index_data['data'][i]['last'],'variation':index_data['data'][i]['variation'],
                                                          'percentChange':index_data['data'][i]['percentChange']}
            df=pd.DataFrame(dict)
            return df


        
        def get_index_constituents_status(symbol):
            url='https://www.nseindia.com/api/equity-stockIndices?index={}'.format(symbol)
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}

            r = requests.get(url,headers=headers)
            data=r.json()
            item = data['data']
            list_=[]
            list_time=[]
            for i in range(len(item)):
                list_time.append(data["timestamp"])
                list_.append(data['data'][i])
            df = pd.DataFrame(list_)
            df_time = pd.DataFrame(list_time,columns=["timestamp"])
            df_combined = pd.concat([df,df_time],axis=1)
            df_combined_sorted =df_combined[["timestamp","symbol","lastPrice","change","pChange","totalTradedVolume"]]
            df_out = df_combined_sorted.sort_values("pChange",ascending=False)
            return df_out

        import requests
        import pandas as pd
        import time
        from bs4 import BeautifulSoup

        df=get_index_status_nse()
        dji=get_index_status_yf()
        df1 =df[["NIFTY 50","NIFTY BANK","INDIA VIX"]]
        df2 = df[[
         'NIFTY 50','NIFTY NEXT 50', 'INDIA VIX', 'NIFTY BANK', 'NIFTY AUTO', 'NIFTY FIN SERVICE', 'NIFTY FMCG',
         'NIFTY IT', 'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY PVT BANK', 'NIFTY REALTY', 'NIFTY HEALTHCARE', 'NIFTY CONSR DURBL', 'NIFTY OIL AND GAS',
         ]]

        df_nifty_out=get_index_constituents_status('NIFTY 50')
        df_banknifty_out=get_index_constituents_status('NIFTY BANK')

        print('\x1b[6;30;42m' + 'Success!' + '\x1b[0m')
        # display(df1.T)
        # display(dji)
        print("Sectoral Indices")
        display(df2.T.sort_values("percentChange",ascending=False).head(5))
        print("BankNifty constituents top-5")
        display(df_banknifty_out.head(5))
        print("Nifty constituents top-5")
        display(df_nifty_out.head(5))
        break
    except:
        print("Site is busy,trying again....")

Site is busy,trying again....
Site is busy,trying again....
Site is busy,trying again....
Success!
Sectoral Indices


,timestamp,last,variation,percentChange
INDIA VIX,10-Oct-2022 09:16:45,20.06,1.25,6.63
NIFTY IT,10-Oct-2022 09:16:45,27485.25,-248.35,-0.9
NIFTY OIL AND GAS,10-Oct-2022 09:16:45,7617.7,-69.95,-0.91
NIFTY HEALTHCARE,10-Oct-2022 09:16:45,8219.45,-76.6,-0.92
NIFTY PHARMA,10-Oct-2022 09:16:45,13009.75,-124.95,-0.95


BankNifty constituents top-5


,timestamp,symbol,lastPrice,change,pChange,totalTradedVolume
1,10-Oct-2022 09:17:15,ICICIBANK,873.40,-9.15,-1.04,291480
2,10-Oct-2022 09:17:15,PNB,36.25,-0.40,-1.09,846928
3,10-Oct-2022 09:17:15,AXISBANK,747.05,-8.65,-1.14,284552
4,10-Oct-2022 09:17:15,SBIN,523.80,-6.40,-1.21,419491
5,10-Oct-2022 09:17:15,AUBANK,582.80,-8.25,-1.40,49903


Nifty constituents top-5


,timestamp,symbol,lastPrice,change,pChange,totalTradedVolume
1,10-Oct-2022 09:16:45,COALINDIA,229.65,-0.05,-0.02,207899
2,10-Oct-2022 09:16:45,POWERGRID,209.20,-0.20,-0.10,312675
3,10-Oct-2022 09:16:45,APOLLOHOSP,4379.00,-15.10,-0.34,8429
4,10-Oct-2022 09:16:45,TCS,3051.80,-13.10,-0.43,182221
5,10-Oct-2022 09:16:45,JSWSTEEL,663.05,-3.00,-0.45,66696


## Losers

In [12]:
while True:
    try:
        print("Sectoral Indices")
        display(df2.T.sort_values("percentChange",ascending=True).head(5))
        print("BankNifty constituents top-5")
        display(df_banknifty_out.sort_values("pChange",ascending=True).head(5))
        print("Nifty constituents top-5")
        display(df_nifty_out.sort_values("pChange",ascending=True).head(5))
        break
    except:
        print("Site is busy,trying again....")

Sectoral Indices


,timestamp,last,variation,percentChange
NIFTY AUTO,10-Oct-2022 09:16:45,12554.45,-207.5,-1.63
NIFTY PVT BANK,10-Oct-2022 09:16:45,20030.75,-316.3,-1.55
NIFTY BANK,10-Oct-2022 09:16:45,38606.5,-571.55,-1.46
NIFTY PSU BANK,10-Oct-2022 09:16:45,2976,-43,-1.42
NIFTY FIN SERVICE,10-Oct-2022 09:16:45,17456.45,-250.55,-1.41


BankNifty constituents top-5


,timestamp,symbol,lastPrice,change,pChange,totalTradedVolume
12,10-Oct-2022 09:17:15,BANDHANBNK,265.45,-9.45,-3.44,468480
11,10-Oct-2022 09:17:15,INDUSINDBK,1188.25,-25.15,-2.07,104982
10,10-Oct-2022 09:17:15,BANKBARODA,132.55,-2.75,-2.03,671070
9,10-Oct-2022 09:17:15,IDFCFIRSTB,51.95,-1.05,-1.98,1902651
8,10-Oct-2022 09:17:15,HDFCBANK,1405.10,-25.70,-1.80,388119


Nifty constituents top-5


,timestamp,symbol,lastPrice,change,pChange,totalTradedVolume
50,10-Oct-2022 09:16:45,TATAMOTORS,398.20,-13.95,-3.38,1970308
49,10-Oct-2022 09:16:45,HEROMOTOCO,2564.50,-60.45,-2.30,71070
48,10-Oct-2022 09:16:45,INDUSINDBK,1188.25,-25.15,-2.07,104982
47,10-Oct-2022 09:16:45,HDFCBANK,1405.10,-25.70,-1.80,388119
46,10-Oct-2022 09:16:45,HDFC,2285.65,-41.25,-1.77,118098
